## I- Récupération des données sur la nature du risque d'inondation 

A partir de l'API TRI Géorisques, nous proposons de récuperer la nature du risque d'inondation pour chaque commune de France Métropolitaine. 

La première cellule permet d'interroger l'API. Elle prend une trentaine de minutes à s'exécuter, comme l'API doit être appelée une fois pour le 36 529 communes françaises. 

Nous avons pris soin de sauvegarder le fichier csv renvoyé par cet appel sous le nom 'flood_risk_data.csv'

In [ ]:
import pandas as pd
import requests

# URL Communes INSEE 
url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"

# Lecture du csv renseignant sur les Communes 
df = pd.read_csv(url)
print("File successfully loaded!")
print(df.head())

# Setup de l'URL de l'API Georisque 
georisques_api_url = "https://georisques.gouv.fr/api/v1/gaspar/tri"

# Fonction qui appelle l'API et récupère les données d'inondation pour une commune 
def get_flood_risk_data(commune_code):
    request_url = f"{georisques_api_url}?code_insee={commune_code}&page=1&page_size=10"
    response = requests.get(request_url)
    if response.status_code == 200:
        json_data = response.json()
        if json_data and 'data' in json_data and len(json_data['data']) > 0:
            risk_data = pd.DataFrame(json_data['data'])
            return risk_data
    return None 

all_risk_data = []

#Boucle pour récupérer les données d'inondation de chaque commune
for index, row in df.iterrows():
    commune_code = row['COM']  #"COM" contient les numéros de commune
    risk_data = get_flood_risk_data(commune_code)
    if risk_data is not None:
        all_risk_data.append(risk_data)

#Concaténation des dataframes 
if all_risk_data:
    combined_risk_data = pd.concat(all_risk_data, ignore_index=True)
    print("Combined flood risk data:")
    print(combined_risk_data.head())
else:
    print("No valid flood risk data found for any commune.")

#Sauvegarde du fichier sous format csv 
combined_risk_data.to_csv("flood_risk_data.csv", index=False)
print("Flood risk data saved to 'flood_risk_data.csv'")


## Commencer à éxécuter les cellules à partir d'ici

In [ ]:
import pandas as pd
import requests
import ast  # Pour convertir une chaîne représentant une liste en liste

# Charger uniquement les colonnes nécessaires
cols_to_load = ['liste_libelle_risque', 'libelle_commune']
data = pd.read_csv('flood_risk_data.csv')

# Convertir la colonne `liste_libelle_risque` en une liste de dictionnaires
data['liste_libelle_risque'] = data['liste_libelle_risque'].apply(ast.literal_eval)

# Séparer les éléments des dictionnaires en colonnes distinctes
expanded_data = data.explode('liste_libelle_risque')  # Sépare chaque élément de la liste dans une nouvelle ligne
expanded_data[['num_risque', 'libelle_risque_long']] = expanded_data['liste_libelle_risque'].apply(pd.Series)

# Supprimer la colonne d'origine après extraction
expanded_data = expanded_data.drop(columns=['liste_libelle_risque'])

# Afficher les premières lignes du DataFrame transformé
expanded_data.head()

Afin de pouvoir réutiliser la dataframe traitée, nous la conservons également sous format csv

In [15]:
#conserver en format csv la dataFrame transformée 
expanded_data.to_csv("Type_inondation_data.csv", index=False)

Le long de ce projet, nous utiliserons la bibliothèque cartiflette afin de construire des cartes de France. L'installation du package est nécessaire à son utilisation : 

In [ ]:
# Importation du package Cartiflette 
!pip install py7zr geopandas openpyxl tqdm s3fs
!pip install PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from cartiflette import carti_download

#Télécharger les données géographiques des communes françaises via Cartiflette
france = carti_download(
    values=["France"],
    crs=4326,
    borders="COMMUNE",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022,
)
print("CRS de france:", france.crs)

# Filtrer les communes métropolitaines pour le fond de carte
france = france.loc[france["INSEE_REG"] > 10]

#Charger les données de risques transformées et associer les risques multiples à chaque commune
risk_data_grouped = expanded_data.groupby('code_insee')['libelle_risque_long'].apply(
    lambda x: ', '.join(x.unique())
).reset_index()

#Fusionner les données géographiques avec les risques
geo_risks = france.merge(risk_data_grouped, left_on='INSEE_COM', right_on='code_insee', how='left')

# 4. Créer la carte
fig, ax = plt.subplots(1, 1, figsize=(12, 15))

# Afficher les limites des communes
france.boundary.plot(ax=ax, linewidth=0.5, color="gray")

# Remplir les communes avec des couleurs basées sur les risques
geo_risks.plot(
    column='libelle_risque_long',  # Colonne contenant les types de risques
    ax=ax,
    legend=True,
    cmap="Set3",  # Palette de couleurs pour différencier les risques
    legend_kwds={'bbox_to_anchor': (1, 1), 'loc': 'upper left'}
)

# Ajouter un titre et améliorer la mise en page
plt.title("Carte des risques par commune en France", fontsize=16)
plt.tight_layout()

# Afficher la carte
plt.show()


Si erreur : AttributeError: module 'urllib3.util' has no attribute 'PROTOCOL_TLS' lors de l'utilisation de cartiflette, 

%pip install --upgrade urllib3

cette première carte nous permet de mettre en évidence les différents types d'évènements pouvant causer une inondation, Parmis eux nous retrouvons : 

In [ ]:
print(expanded_data['libelle_risque_long'].unique())

Cette première carte nous permet également de pouvoir mettre en évidence les zones d'inondation exacerbées. Le Sud Ouest de la France semble être une zone géographique française sinistrée par les inondations. 